In [ ]:
!pip install "git+https://github.com/google/uncertainty-baselines.git#egg=uncertainty_baselines"

  Cloning https://github.com/google/uncertainty-baselines.git to /tmp/pip-install-x9ksb117/uncertainty-baselines
  Running command git clone -q https://github.com/google/uncertainty-baselines.git /tmp/pip-install-x9ksb117/uncertainty-baselines
     |████████████████████████████████| 92kB 3.6MB/s 
     |████████████████████████████████| 5.5MB 27.4MB/s 
     |████████████████████████████████| 454.7MB 34kB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
     |████████████████████████████████| 471kB 20.1MB/s 
     |████████████████████████████████| 1.3MB 31.1MB/s 
  Created wheel for uncertainty-baselines: filename=uncertainty_baselines-0.0.7-cp37-none-any.whl size=276276 sha256=ad3d2ddc2a4889bea916ed7045ee757e5f3f120f8ed62729f9dfccf4790625ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-d_fiazgr/wheels/5b/77/8b/8954f4644619426772bf3f47cb2246de4be5409186b43b0264
Successfully built uncertainty-baselines
ERROR: tensorflow 2.5.0 has requirement keras-nightly~=2.5.0.dev, but you

In [ ]:
pip install "git+https://github.com/google-research/robustness_metrics.git#egg=robustness_metrics"

  Cloning https://github.com/google-research/robustness_metrics.git to /tmp/pip-install-q9uxf9qn/robustness-metrics
  Running command git clone -q https://github.com/google-research/robustness_metrics.git /tmp/pip-install-q9uxf9qn/robustness-metrics
     |████████████████████████████████| 5.5MB 4.2MB/s 
     |████████████████████████████████| 4.2MB 33.0MB/s 
  Created wheel for robustness-metrics: filename=robustness_metrics-0.0.1-cp37-none-any.whl size=101438 sha256=298884611e89ccb773adc15a899d17a24c676adc44aaf84b5403fb531a96f35e
  Stored in directory: /tmp/pip-ephem-wheel-cache-knk34_6o/wheels/f1/09/d6/441f73c886118e9f7afb6c7c15f0273279567036d7390e345d
Successfully built robustness-metrics
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.38.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement keras-nightly~=2.5.0.dev, but you'll have keras-nightly 2.6.0.dev2021061500 which is incompatible.
  Found existing installation: grpcio 1.34.1
    Un

In [ ]:
pip install "git+https://github.com/google/edward2"

  Cloning https://github.com/google/edward2 to /tmp/pip-req-build-ryih37lf
  Running command git clone -q https://github.com/google/edward2 /tmp/pip-req-build-ryih37lf
  Created wheel for edward2: filename=edward2-0.0.3-cp37-none-any.whl size=168433 sha256=83e47ff2018637d228e0dda544e773d706991701ea8516aa654cdeb4f68a24a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-x8hudufv/wheels/f9/fb/ed/96a1b2e305c0df592e14b8a7bf576b09f1540542771856eb15
Successfully built edward2


In [ ]:
!pip install tensorflow_addons
!pip install tensorflow_datasets --upgrade

     |████████████████████████████████| 686kB 4.1MB/s 
     |████████████████████████████████| 3.9MB 4.2MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


#시작

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow_datasets as tfds

dataset_builder = tfds.builder('mnist')
dataset_builder.download_and_prepare()


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
dataset_builder = tfds.builder('fashion_mnist')
dataset_builder.download_and_prepare()

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


# utils

In [ ]:
import numpy as np
import scipy


def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])


def brier_score(y, p):
  return np.mean(np.power(p - y, 2))


def calibration(y, p_mean, num_bins=10):
  class_pred = np.argmax(p_mean, axis=1)
  conf = np.max(p_mean, axis=1)
  y = np.argmax(y, axis=1)
  acc_tab = np.zeros(num_bins)  
  mean_conf = np.zeros(num_bins)  
  nb_items_bin = np.zeros(num_bins) 
  tau_tab = np.linspace(0, 1, num_bins+1)  
  for i in np.arange(num_bins):  
    sec = (tau_tab[i + 1] > conf) & (conf >= tau_tab[i])
    nb_items_bin[i] = np.sum(sec)  
    class_pred_sec, y_sec = class_pred[sec], y[sec]
    mean_conf[i] = np.mean(conf[sec]) if nb_items_bin[i] > 0 else np.nan
    acc_tab[i] = np.mean(
        class_pred_sec == y_sec) if nb_items_bin[i] > 0 else np.nan

  mean_conf = mean_conf[nb_items_bin > 0]
  acc_tab = acc_tab[nb_items_bin > 0]
  nb_items_bin = nb_items_bin[nb_items_bin > 0]

  ece = np.average(
      np.absolute(mean_conf - acc_tab),
      weights=nb_items_bin.astype(np.float) / np.sum(nb_items_bin))
  mce = np.max(np.absolute(mean_conf - acc_tab))
  return ece, mce


def ensemble_metrics(x,
                     y,
                     model,
                     log_likelihood_fn,
                     n_samples=1,
                     weight_files=None):
  if weight_files is None:
    ensemble_logprobs = [log_likelihood_fn([x, y])[0] for _ in range(n_samples)]
    metric_values = [model.evaluate(x, y, verbose=0)
                     for _ in range(n_samples)]
    ensemble_logits = [log_likelihood_fn([x, y])[1] for _ in range(n_samples)]
  else:
    ensemble_logprobs = []
    metric_values = []
    ensemble_logits = []
    for filename in weight_files:
      model.load_weights(filename)
      ensemble_logprobs.extend([log_likelihood_fn([x, y])[0]
                                for _ in range(n_samples)])
      ensemble_logits.extend([log_likelihood_fn([x, y])[1]
                              for _ in range(n_samples)])
      metric_values.extend([model.evaluate(x, y, verbose=0)
                            for _ in range(n_samples)])

  metric_values = np.mean(np.array(metric_values), axis=0)
  results = {}
  for m, name in zip(metric_values, model.metrics_names):
    results[name] = m

  ensemble_logprobs = np.array(ensemble_logprobs)
  probabilistic_log_likelihood = np.mean(
      scipy.special.logsumexp(
          np.sum(ensemble_logprobs, axis=2)
          if len(ensemble_logprobs.shape) > 2 else ensemble_logprobs,
          b=1. / ensemble_logprobs.shape[0],
          axis=0),
      axis=0)
  results['probabilistic_log_likelihood'] = probabilistic_log_likelihood

  ensemble_logits = np.array(ensemble_logits)
  probs = np.mean(scipy.special.softmax(ensemble_logits, axis=2), axis=0)
  class_pred = np.argmax(probs, axis=1)
  probabilistic_accuracy = np.mean(np.equal(y, class_pred))
  results['probabilistic_accuracy'] = probabilistic_accuracy
  results['ece'], results['mce'] = calibration(
      one_hot(y, probs.shape[1]), probs)
  results['brier_score'] = brier_score(one_hot(y, probs.shape[1]), probs)
  return results

# mnist Single

In [ ]:
import os
from absl import app
from absl import flags
from absl import logging

import edward2 as ed
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import uncertainty_baselines as ub
#import utils  # local file import

flags.DEFINE_string("f", "", "kernel")

flags.DEFINE_enum('dataset', 'mnist',
                  enum_values=['mnist', 'fashion_mnist'],
                  help='Name of the image dataset.')
flags.DEFINE_integer('ensemble_size', 1, 'Number of ensemble members.')
flags.DEFINE_boolean('bootstrap', False,
                     'Sample the training set for bootstrapping.')
flags.DEFINE_integer('training_steps', 5000, 'Training steps.')
flags.DEFINE_integer('batch_size', 256, 'Batch size.')
flags.DEFINE_float('learning_rate', 0.001, 'Learning rate.')
flags.DEFINE_integer('validation_freq', 5, 'Validation frequency in steps.')
flags.DEFINE_string('output_dir', '/content/gdrive/My Drive/tmp/mnist/single',
                    'The directory where the model weights and '
                    'training/evaluation summaries are stored.')
flags.DEFINE_integer('seed', 0, 'Random seed.')
flags.DEFINE_bool('use_gpu', True, 'Whether to run on GPU or otherwise TPU.')
flags.DEFINE_integer('num_cores', 1, 'Number of TPU cores or number of GPUs.')
FLAGS = flags.FLAGS


def lenet5(input_shape, num_classes):
  """Builds LeNet5."""
  inputs = tf.keras.layers.Input(shape=input_shape)
  conv1 = tf.keras.layers.Conv2D(6,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(inputs)
  pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv1)
  conv2 = tf.keras.layers.Conv2D(16,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(pool1)
  pool2 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv2)
  conv3 = tf.keras.layers.Conv2D(120,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation=tf.nn.relu)(pool2)
  flatten = tf.keras.layers.Flatten()(conv3)
  dense1 = tf.keras.layers.Dense(84, activation=tf.nn.relu)(flatten)
  logits = tf.keras.layers.Dense(num_classes)(dense1)
  outputs = tf.keras.layers.Lambda(lambda x: ed.Categorical(logits=x))(logits)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def main(argv):
  del argv  # unused arg
#   if not FLAGS.use_gpu:
#     raise ValueError('Only GPU is currently supported.')
#   if FLAGS.num_cores > 1:
#     raise ValueError('Only a single accelerator is currently supported.')
  np.random.seed(FLAGS.seed)
  tf.random.set_seed(FLAGS.seed)
  tf.io.gfile.makedirs(FLAGS.output_dir)

  if FLAGS.dataset == 'mnist':
    dataset_builder_class = ub.datasets.MnistDataset
  else:
    dataset_builder_class = ub.datasets.FashionMnistDataset
  n_train = 50000
  train_dataset = next(dataset_builder_class(
      'train').load(batch_size=n_train).as_numpy_iterator())
  x_train = train_dataset['features']
  y_train = train_dataset['labels']
  test_dataset = next(dataset_builder_class(
      'test').load(batch_size=10000).as_numpy_iterator())
  x_test = test_dataset['features']
  y_test = test_dataset['labels']
  num_classes = int(np.amax(y_train)) + 1

  tf1.disable_v2_behavior()

  ensemble_filenames = []
  for i in range(FLAGS.ensemble_size):
    model = lenet5(x_train.shape[1:], num_classes)

    def negative_log_likelihood(y, rv_y):
      del rv_y  # unused arg
      return -model.output.distribution.log_prob(tf.squeeze(y))  # pylint: disable=cell-var-from-loop

    def accuracy(y_true, y_sample):
      del y_sample  # unused arg
      return tf.equal(
          tf.argmax(input=model.output.distribution.logits, axis=1),  # pylint: disable=cell-var-from-loop
          tf.cast(tf.squeeze(y_true), tf.int64))

    def log_likelihood(y_true, y_sample):
      del y_sample  # unused arg
      return model.output.distribution.log_prob(tf.squeeze(y_true))  # pylint: disable=cell-var-from-loop

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=FLAGS.learning_rate),
        loss=negative_log_likelihood,
        metrics=[log_likelihood, accuracy])
    member_dir = os.path.join(FLAGS.output_dir, 'member_' + str(i))
    tensorboard = tf1.keras.callbacks.TensorBoard(
        log_dir=member_dir,
        update_freq=FLAGS.batch_size * FLAGS.validation_freq)

    if FLAGS.bootstrap:
      inds = np.random.choice(n_train, n_train, replace=True)
      x_sampled = x_train[inds]
      y_sampled = y_train[inds]

    model.fit(
        x=x_train if not FLAGS.bootstrap else x_sampled,
        y=y_train if not FLAGS.bootstrap else y_sampled,
        batch_size=FLAGS.batch_size,
        epochs=(FLAGS.batch_size * FLAGS.training_steps) // n_train,
        validation_data=(x_test, y_test),
        validation_freq=max(
            (FLAGS.validation_freq * FLAGS.batch_size) // n_train, 1),
        verbose=1,
        callbacks=[tensorboard])

    member_filename = os.path.join(member_dir, 'model.weights')
    ensemble_filenames.append(member_filename)
    model.save_weights(member_filename)

  labels = tf.keras.layers.Input(shape=y_train.shape[1:])
  ll = tf.keras.backend.function([model.input, labels], [
      model.output.distribution.log_prob(tf.squeeze(labels)),
      model.output.distribution.logits,
  ])

  ensemble_metrics_vals = {
      'train': ensemble_metrics(
          x_train, y_train, model, ll, weight_files=ensemble_filenames),
      'test': ensemble_metrics(
          x_test, y_test, model, ll, weight_files=ensemble_filenames),
  }

  for split, metrics in ensemble_metrics_vals.items():
    logging.info(split)
    for metric_name in metrics:
      logging.info('%s: %s', metric_name, metrics[metric_name])

if __name__ == '__main__':
  app.run(main)

/usr/local/lib/python3.7/dist-packages/edward2/__init__.py:34: UserWarning: JAX backend for Edward2 is not available.
  warnings.warn("JAX backend for Edward2 is not available.")
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/models.py:39: UserWarning: Skipped due to ImportError: No module named 'official'
  warnings.warn(f'Skipped due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:67: UserWarning: Skipped ViT models due to ImportError: No module named 'flax'
  warnings.warn(f'Skipped ViT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:76: UserWarning: Skipped BERT models due to ImportError: No module named 'official'
  warnings.warn(f'Skipped BERT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:84: UserWarning: Skipped MIMO models due to ImportError: No module named 'edward2.experim

Instructions for updating:
non-resource variables are not supported in the long term


W0615 07:02:16.204505 140189709981568 deprecation.py:345] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - ETA: 0s - loss: 0.3211 - log_likelihood: -0.3211 - accuracy: 0.9011

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 43s 863us/sample - loss: 0.3211 - log_likelihood: -0.3211 - accuracy: 0.9011 - val_loss: 0.0768 - val_log_likelihood: -0.0768 - val_accuracy: 0.9772
Epoch 2/25
50000/50000 [==============================] - 42s 846us/sample - loss: 0.0781 - log_likelihood: -0.0781 - accuracy: 0.9757 - val_loss: 0.0837 - val_log_likelihood: -0.0837 - val_accuracy: 0.9734
Epoch 3/25
50000/50000 [==============================] - 42s 839us/sample - loss: 0.0562 - log_likelihood: -0.0562 - accuracy: 0.9826 - val_loss: 0.0489 - val_log_likelihood: -0.0489 - val_accuracy: 0.9838
Epoch 4/25
50000/50000 [==============================] - 42s 841us/sample - loss: 0.0409 - log_likelihood: -0.0409 - accuracy: 0.9872 - val_loss: 0.0367 - val_log_likelihood: -0.0367 - val_accuracy: 0.9869
Epoch 5/25
50000/50000 [==============================] - 42s 840us/sample - loss: 0.0321 - log_likelihood: -0.0321 - accuracy: 0.9897 - val_loss: 0.0378 - val_log_likelihood: -0.0378

I0615 07:20:55.086890 140189709981568 <ipython-input-9-0a7738d57fce>:172] train
I0615 07:20:55.088428 140189709981568 <ipython-input-9-0a7738d57fce>:174] loss: 0.0031167311582124147
I0615 07:20:55.089649 140189709981568 <ipython-input-9-0a7738d57fce>:174] log_likelihood: -0.003116729436442256
I0615 07:20:55.095212 140189709981568 <ipython-input-9-0a7738d57fce>:174] accuracy: 0.9990400075912476
I0615 07:20:55.096665 140189709981568 <ipython-input-9-0a7738d57fce>:174] probabilistic_log_likelihood: -0.003116731363973938
I0615 07:20:55.098550 140189709981568 <ipython-input-9-0a7738d57fce>:174] probabilistic_accuracy: 0.99904
I0615 07:20:55.103627 140189709981568 <ipython-input-9-0a7738d57fce>:174] ece: 0.0011740893125534047
I0615 07:20:55.106904 140189709981568 <ipython-input-9-0a7738d57fce>:174] mce: 0.10579868555068972
I0615 07:20:55.108407 140189709981568 <ipython-input-9-0a7738d57fce>:174] brier_score: 0.00015836751674568994
I0615 07:20:55.110226 140189709981568 <ipython-input-9-0a7738

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# mnist ensemble(size=5)

In [ ]:
import os
from absl import app
from absl import flags
from absl import logging

import edward2 as ed
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import uncertainty_baselines as ub
#import utils  # local file import

flags.DEFINE_string("f", "", "kernel")

flags.DEFINE_enum('dataset', 'mnist',
                  enum_values=['mnist', 'fashion_mnist'],
                  help='Name of the image dataset.')
flags.DEFINE_integer('ensemble_size', 5, 'Number of ensemble members.')
flags.DEFINE_boolean('bootstrap', False,
                     'Sample the training set for bootstrapping.')
flags.DEFINE_integer('training_steps', 5000, 'Training steps.')
flags.DEFINE_integer('batch_size', 256, 'Batch size.')
flags.DEFINE_float('learning_rate', 0.001, 'Learning rate.')
flags.DEFINE_integer('validation_freq', 5, 'Validation frequency in steps.')
flags.DEFINE_string('output_dir', '/content/gdrive/My Drive/tmp/mnist/ensemble',
                    'The directory where the model weights and '
                    'training/evaluation summaries are stored.')
flags.DEFINE_integer('seed', 0, 'Random seed.')
flags.DEFINE_bool('use_gpu', True, 'Whether to run on GPU or otherwise TPU.')
flags.DEFINE_integer('num_cores', 1, 'Number of TPU cores or number of GPUs.')
FLAGS = flags.FLAGS


def lenet5(input_shape, num_classes):
  """Builds LeNet5."""
  inputs = tf.keras.layers.Input(shape=input_shape)
  conv1 = tf.keras.layers.Conv2D(6,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(inputs)
  pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv1)
  conv2 = tf.keras.layers.Conv2D(16,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(pool1)
  pool2 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv2)
  conv3 = tf.keras.layers.Conv2D(120,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation=tf.nn.relu)(pool2)
  flatten = tf.keras.layers.Flatten()(conv3)
  dense1 = tf.keras.layers.Dense(84, activation=tf.nn.relu)(flatten)
  logits = tf.keras.layers.Dense(num_classes)(dense1)
  outputs = tf.keras.layers.Lambda(lambda x: ed.Categorical(logits=x))(logits)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def main(argv):
  del argv  # unused arg
#   if not FLAGS.use_gpu:
#     raise ValueError('Only GPU is currently supported.')
#   if FLAGS.num_cores > 1:
#     raise ValueError('Only a single accelerator is currently supported.')
  np.random.seed(FLAGS.seed)
  tf.random.set_seed(FLAGS.seed)
  tf.io.gfile.makedirs(FLAGS.output_dir)

  if FLAGS.dataset == 'mnist':
    dataset_builder_class = ub.datasets.MnistDataset
  else:
    dataset_builder_class = ub.datasets.FashionMnistDataset
  n_train = 50000
  train_dataset = next(dataset_builder_class(
      'train').load(batch_size=n_train).as_numpy_iterator())
  x_train = train_dataset['features']
  y_train = train_dataset['labels']
  test_dataset = next(dataset_builder_class(
      'test').load(batch_size=10000).as_numpy_iterator())
  x_test = test_dataset['features']
  y_test = test_dataset['labels']
  num_classes = int(np.amax(y_train)) + 1

  tf1.disable_v2_behavior()

  ensemble_filenames = []
  for i in range(FLAGS.ensemble_size):
    model = lenet5(x_train.shape[1:], num_classes)

    def negative_log_likelihood(y, rv_y):
      del rv_y  # unused arg
      return -model.output.distribution.log_prob(tf.squeeze(y))  # pylint: disable=cell-var-from-loop

    def accuracy(y_true, y_sample):
      del y_sample  # unused arg
      return tf.equal(
          tf.argmax(input=model.output.distribution.logits, axis=1),  # pylint: disable=cell-var-from-loop
          tf.cast(tf.squeeze(y_true), tf.int64))

    def log_likelihood(y_true, y_sample):
      del y_sample  # unused arg
      return model.output.distribution.log_prob(tf.squeeze(y_true))  # pylint: disable=cell-var-from-loop

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=FLAGS.learning_rate),
        loss=negative_log_likelihood,
        metrics=[log_likelihood, accuracy])
    member_dir = os.path.join(FLAGS.output_dir, 'member_' + str(i))
    tensorboard = tf1.keras.callbacks.TensorBoard(
        log_dir=member_dir,
        update_freq=FLAGS.batch_size * FLAGS.validation_freq)

    if FLAGS.bootstrap:
      inds = np.random.choice(n_train, n_train, replace=True)
      x_sampled = x_train[inds]
      y_sampled = y_train[inds]

    model.fit(
        x=x_train if not FLAGS.bootstrap else x_sampled,
        y=y_train if not FLAGS.bootstrap else y_sampled,
        batch_size=FLAGS.batch_size,
        epochs=(FLAGS.batch_size * FLAGS.training_steps) // n_train,
        validation_data=(x_test, y_test),
        validation_freq=max(
            (FLAGS.validation_freq * FLAGS.batch_size) // n_train, 1),
        verbose=1,
        callbacks=[tensorboard])

    member_filename = os.path.join(member_dir, 'model.weights')
    ensemble_filenames.append(member_filename)
    model.save_weights(member_filename)

  labels = tf.keras.layers.Input(shape=y_train.shape[1:])
  ll = tf.keras.backend.function([model.input, labels], [
      model.output.distribution.log_prob(tf.squeeze(labels)),
      model.output.distribution.logits,
  ])

  ensemble_metrics_vals = {
      'train': ensemble_metrics(
          x_train, y_train, model, ll, weight_files=ensemble_filenames),
      'test': ensemble_metrics(
          x_test, y_test, model, ll, weight_files=ensemble_filenames),
  }

  for split, metrics in ensemble_metrics_vals.items():
    logging.info(split)
    for metric_name in metrics:
      logging.info('%s: %s', metric_name, metrics[metric_name])

if __name__ == '__main__':
  app.run(main)

/usr/local/lib/python3.7/dist-packages/edward2/__init__.py:34: UserWarning: JAX backend for Edward2 is not available.
  warnings.warn("JAX backend for Edward2 is not available.")
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/models.py:39: UserWarning: Skipped due to ImportError: No module named 'official'
  warnings.warn(f'Skipped due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:67: UserWarning: Skipped ViT models due to ImportError: No module named 'flax'
  warnings.warn(f'Skipped ViT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:76: UserWarning: Skipped BERT models due to ImportError: No module named 'official'
  warnings.warn(f'Skipped BERT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:84: UserWarning: Skipped MIMO models due to ImportError: No module named 'edward2.experim

Instructions for updating:
non-resource variables are not supported in the long term


W0615 07:23:13.509845 139718465394560 deprecation.py:345] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - ETA: 0s - loss: 0.3361 - log_likelihood: -0.3361 - accuracy: 0.8991

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 43s 866us/sample - loss: 0.3361 - log_likelihood: -0.3361 - accuracy: 0.8991 - val_loss: 0.0830 - val_log_likelihood: -0.0830 - val_accuracy: 0.9738
Epoch 2/25
50000/50000 [==============================] - 44s 885us/sample - loss: 0.0754 - log_likelihood: -0.0754 - accuracy: 0.9767 - val_loss: 0.0534 - val_log_likelihood: -0.0534 - val_accuracy: 0.9826
Epoch 3/25
50000/50000 [==============================] - 43s 867us/sample - loss: 0.0543 - log_likelihood: -0.0543 - accuracy: 0.9831 - val_loss: 0.0556 - val_log_likelihood: -0.0556 - val_accuracy: 0.9811
Epoch 4/25
50000/50000 [==============================] - 45s 890us/sample - loss: 0.0398 - log_likelihood: -0.0398 - accuracy: 0.9875 - val_loss: 0.0400 - val_log_likelihood: -0.0400 - val_accuracy: 0.9860
Epoch 5/25
50000/50000 [==============================] - 44s 887us/sample - loss: 0.0303 - log_likelihood: -0.0303 - accuracy: 0.9904 - val_loss: 0.0351 - val_log_likelihood: -0.0351

I0615 08:59:07.802929 139718465394560 <ipython-input-11-2e0dc3841369>:172] train
I0615 08:59:07.804318 139718465394560 <ipython-input-11-2e0dc3841369>:174] loss: 0.004329791438820045
I0615 08:59:07.809198 139718465394560 <ipython-input-11-2e0dc3841369>:174] log_likelihood: -0.0043297896161675455
I0615 08:59:07.811261 139718465394560 <ipython-input-11-2e0dc3841369>:174] accuracy: 0.9984880089759827
I0615 08:59:07.812906 139718465394560 <ipython-input-11-2e0dc3841369>:174] probabilistic_log_likelihood: -0.0026553961616633307
I0615 08:59:07.814771 139718465394560 <ipython-input-11-2e0dc3841369>:174] probabilistic_accuracy: 0.99996
I0615 08:59:07.816530 139718465394560 <ipython-input-11-2e0dc3841369>:174] ece: 0.005293247145075939
I0615 08:59:07.817411 139718465394560 <ipython-input-11-2e0dc3841369>:174] mce: 0.3094515999158224
I0615 08:59:07.818377 139718465394560 <ipython-input-11-2e0dc3841369>:174] brier_score: 6.42382572220761e-05
I0615 08:59:07.819506 139718465394560 <ipython-input-11

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# fashion single

In [ ]:
import os
from absl import app
from absl import flags
from absl import logging

import edward2 as ed
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import uncertainty_baselines as ub
#import utils  # local file import

flags.DEFINE_string("f", "", "kernel")

flags.DEFINE_enum('dataset', 'fashion_mnist',
                  enum_values=['mnist', 'fashion_mnist'],
                  help='Name of the image dataset.')
flags.DEFINE_integer('ensemble_size', 1, 'Number of ensemble members.')
flags.DEFINE_boolean('bootstrap', False,
                     'Sample the training set for bootstrapping.')
flags.DEFINE_integer('training_steps', 5000, 'Training steps.')
flags.DEFINE_integer('batch_size', 256, 'Batch size.')
flags.DEFINE_float('learning_rate', 0.001, 'Learning rate.')
flags.DEFINE_integer('validation_freq', 5, 'Validation frequency in steps.')
flags.DEFINE_string('output_dir', '/content/gdrive/My Drive/tmp/fashion/single',
                    'The directory where the model weights and '
                    'training/evaluation summaries are stored.')
flags.DEFINE_integer('seed', 0, 'Random seed.')
flags.DEFINE_bool('use_gpu', True, 'Whether to run on GPU or otherwise TPU.')
flags.DEFINE_integer('num_cores', 1, 'Number of TPU cores or number of GPUs.')
FLAGS = flags.FLAGS


def lenet5(input_shape, num_classes):
  """Builds LeNet5."""
  inputs = tf.keras.layers.Input(shape=input_shape)
  conv1 = tf.keras.layers.Conv2D(6,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(inputs)
  pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv1)
  conv2 = tf.keras.layers.Conv2D(16,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(pool1)
  pool2 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv2)
  conv3 = tf.keras.layers.Conv2D(120,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation=tf.nn.relu)(pool2)
  flatten = tf.keras.layers.Flatten()(conv3)
  dense1 = tf.keras.layers.Dense(84, activation=tf.nn.relu)(flatten)
  logits = tf.keras.layers.Dense(num_classes)(dense1)
  outputs = tf.keras.layers.Lambda(lambda x: ed.Categorical(logits=x))(logits)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def main(argv):
  del argv  # unused arg
#   if not FLAGS.use_gpu:
#     raise ValueError('Only GPU is currently supported.')
#   if FLAGS.num_cores > 1:
#     raise ValueError('Only a single accelerator is currently supported.')
  np.random.seed(FLAGS.seed)
  tf.random.set_seed(FLAGS.seed)
  tf.io.gfile.makedirs(FLAGS.output_dir)

  if FLAGS.dataset == 'mnist':
    dataset_builder_class = ub.datasets.MnistDataset
  else:
    dataset_builder_class = ub.datasets.FashionMnistDataset
  n_train = 50000
  train_dataset = next(dataset_builder_class(
      'train').load(batch_size=n_train).as_numpy_iterator())
  x_train = train_dataset['features']
  y_train = train_dataset['labels']
  test_dataset = next(dataset_builder_class(
      'test').load(batch_size=10000).as_numpy_iterator())
  x_test = test_dataset['features']
  y_test = test_dataset['labels']
  num_classes = int(np.amax(y_train)) + 1

  tf1.disable_v2_behavior()

  ensemble_filenames = []
  for i in range(FLAGS.ensemble_size):
    model = lenet5(x_train.shape[1:], num_classes)

    def negative_log_likelihood(y, rv_y):
      del rv_y  # unused arg
      return -model.output.distribution.log_prob(tf.squeeze(y))  # pylint: disable=cell-var-from-loop

    def accuracy(y_true, y_sample):
      del y_sample  # unused arg
      return tf.equal(
          tf.argmax(input=model.output.distribution.logits, axis=1),  # pylint: disable=cell-var-from-loop
          tf.cast(tf.squeeze(y_true), tf.int64))

    def log_likelihood(y_true, y_sample):
      del y_sample  # unused arg
      return model.output.distribution.log_prob(tf.squeeze(y_true))  # pylint: disable=cell-var-from-loop

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=FLAGS.learning_rate),
        loss=negative_log_likelihood,
        metrics=[log_likelihood, accuracy])
    member_dir = os.path.join(FLAGS.output_dir, 'member_' + str(i))
    tensorboard = tf1.keras.callbacks.TensorBoard(
        log_dir=member_dir,
        update_freq=FLAGS.batch_size * FLAGS.validation_freq)

    if FLAGS.bootstrap:
      inds = np.random.choice(n_train, n_train, replace=True)
      x_sampled = x_train[inds]
      y_sampled = y_train[inds]

    model.fit(
        x=x_train if not FLAGS.bootstrap else x_sampled,
        y=y_train if not FLAGS.bootstrap else y_sampled,
        batch_size=FLAGS.batch_size,
        epochs=(FLAGS.batch_size * FLAGS.training_steps) // n_train,
        validation_data=(x_test, y_test),
        validation_freq=max(
            (FLAGS.validation_freq * FLAGS.batch_size) // n_train, 1),
        verbose=1,
        callbacks=[tensorboard])

    member_filename = os.path.join(member_dir, 'model.weights')
    ensemble_filenames.append(member_filename)
    model.save_weights(member_filename)

  labels = tf.keras.layers.Input(shape=y_train.shape[1:])
  ll = tf.keras.backend.function([model.input, labels], [
      model.output.distribution.log_prob(tf.squeeze(labels)),
      model.output.distribution.logits,
  ])

  ensemble_metrics_vals = {
      'train': ensemble_metrics(
          x_train, y_train, model, ll, weight_files=ensemble_filenames),
      'test': ensemble_metrics(
          x_test, y_test, model, ll, weight_files=ensemble_filenames),
  }

  for split, metrics in ensemble_metrics_vals.items():
    logging.info(split)
    for metric_name in metrics:
      logging.info('%s: %s', metric_name, metrics[metric_name])

if __name__ == '__main__':
  app.run(main)

/usr/local/lib/python3.7/dist-packages/edward2/__init__.py:34: UserWarning: JAX backend for Edward2 is not available.
  warnings.warn("JAX backend for Edward2 is not available.")
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/models.py:39: UserWarning: Skipped due to ImportError: No module named 'official'
  warnings.warn(f'Skipped due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:67: UserWarning: Skipped ViT models due to ImportError: No module named 'flax'
  warnings.warn(f'Skipped ViT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:76: UserWarning: Skipped BERT models due to ImportError: No module named 'official'
  warnings.warn(f'Skipped BERT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:84: UserWarning: Skipped MIMO models due to ImportError: No module named 'edward2.experim

Instructions for updating:
non-resource variables are not supported in the long term


W0615 09:25:15.997740 140085224408960 deprecation.py:345] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - ETA: 0s - loss: 0.6391 - log_likelihood: -0.6391 - accuracy: 0.7701

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 40s 799us/sample - loss: 0.6391 - log_likelihood: -0.6391 - accuracy: 0.7701 - val_loss: 0.4403 - val_log_likelihood: -0.4403 - val_accuracy: 0.8453
Epoch 2/25
50000/50000 [==============================] - 39s 786us/sample - loss: 0.3886 - log_likelihood: -0.3886 - accuracy: 0.8613 - val_loss: 0.3816 - val_log_likelihood: -0.3816 - val_accuracy: 0.8638
Epoch 3/25
50000/50000 [==============================] - 39s 786us/sample - loss: 0.3323 - log_likelihood: -0.3323 - accuracy: 0.8801 - val_loss: 0.3650 - val_log_likelihood: -0.3650 - val_accuracy: 0.8717
Epoch 4/25
50000/50000 [==============================] - 39s 785us/sample - loss: 0.3008 - log_likelihood: -0.3008 - accuracy: 0.8919 - val_loss: 0.3377 - val_log_likelihood: -0.3377 - val_accuracy: 0.8799
Epoch 5/25
50000/50000 [==============================] - 39s 788us/sample - loss: 0.2724 - log_likelihood: -0.2724 - accuracy: 0.9005 - val_loss: 0.3062 - val_log_likelihood: -0.3062

I0615 09:42:28.103286 140085224408960 <ipython-input-9-75706e3151b0>:172] train
I0615 09:42:28.104501 140085224408960 <ipython-input-9-75706e3151b0>:174] loss: 0.040563182377666235
I0615 09:42:28.106687 140085224408960 <ipython-input-9-75706e3151b0>:174] log_likelihood: -0.04056317359209061
I0615 09:42:28.108397 140085224408960 <ipython-input-9-75706e3151b0>:174] accuracy: 0.9870200157165527
I0615 09:42:28.110142 140085224408960 <ipython-input-9-75706e3151b0>:174] probabilistic_log_likelihood: -0.04056318243470941
I0615 09:42:28.111544 140085224408960 <ipython-input-9-75706e3151b0>:174] probabilistic_accuracy: 0.98702
I0615 09:42:28.112979 140085224408960 <ipython-input-9-75706e3151b0>:174] ece: 0.013919411323119735
I0615 09:42:28.114476 140085224408960 <ipython-input-9-75706e3151b0>:174] mce: 0.7035193145275116
I0615 09:42:28.115325 140085224408960 <ipython-input-9-75706e3151b0>:174] brier_score: 0.002084522260507131
I0615 09:42:28.116126 140085224408960 <ipython-input-9-75706e3151b0>

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# fashion ensemble(size=5)

In [ ]:
import os
from absl import app
from absl import flags
from absl import logging

import edward2 as ed
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import uncertainty_baselines as ub
#import utils  # local file import

flags.DEFINE_string("f", "", "kernel")

flags.DEFINE_enum('dataset', 'fashion_mnist',
                  enum_values=['mnist', 'fashion_mnist'],
                  help='Name of the image dataset.')
flags.DEFINE_integer('ensemble_size', 5, 'Number of ensemble members.')
flags.DEFINE_boolean('bootstrap', False,
                     'Sample the training set for bootstrapping.')
flags.DEFINE_integer('training_steps', 5000, 'Training steps.')
flags.DEFINE_integer('batch_size', 256, 'Batch size.')
flags.DEFINE_float('learning_rate', 0.001, 'Learning rate.')
flags.DEFINE_integer('validation_freq', 5, 'Validation frequency in steps.')
flags.DEFINE_string('output_dir', '/content/gdrive/My Drive/tmp/fashion/ensemble',
                    'The directory where the model weights and '
                    'training/evaluation summaries are stored.')
flags.DEFINE_integer('seed', 0, 'Random seed.')
flags.DEFINE_bool('use_gpu', True, 'Whether to run on GPU or otherwise TPU.')
flags.DEFINE_integer('num_cores', 1, 'Number of TPU cores or number of GPUs.')
FLAGS = flags.FLAGS


def lenet5(input_shape, num_classes):
  """Builds LeNet5."""
  inputs = tf.keras.layers.Input(shape=input_shape)
  conv1 = tf.keras.layers.Conv2D(6,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(inputs)
  pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv1)
  conv2 = tf.keras.layers.Conv2D(16,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation='relu')(pool1)
  pool2 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2],
                                       strides=[2, 2],
                                       padding='SAME')(conv2)
  conv3 = tf.keras.layers.Conv2D(120,
                                 kernel_size=5,
                                 padding='SAME',
                                 activation=tf.nn.relu)(pool2)
  flatten = tf.keras.layers.Flatten()(conv3)
  dense1 = tf.keras.layers.Dense(84, activation=tf.nn.relu)(flatten)
  logits = tf.keras.layers.Dense(num_classes)(dense1)
  outputs = tf.keras.layers.Lambda(lambda x: ed.Categorical(logits=x))(logits)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def main(argv):
  del argv  # unused arg
#   if not FLAGS.use_gpu:
#     raise ValueError('Only GPU is currently supported.')
#   if FLAGS.num_cores > 1:
#     raise ValueError('Only a single accelerator is currently supported.')
  np.random.seed(FLAGS.seed)
  tf.random.set_seed(FLAGS.seed)
  tf.io.gfile.makedirs(FLAGS.output_dir)

  if FLAGS.dataset == 'mnist':
    dataset_builder_class = ub.datasets.MnistDataset
  else:
    dataset_builder_class = ub.datasets.FashionMnistDataset
  n_train = 50000
  train_dataset = next(dataset_builder_class(
      'train').load(batch_size=n_train).as_numpy_iterator())
  x_train = train_dataset['features']
  y_train = train_dataset['labels']
  test_dataset = next(dataset_builder_class(
      'test').load(batch_size=10000).as_numpy_iterator())
  x_test = test_dataset['features']
  y_test = test_dataset['labels']
  num_classes = int(np.amax(y_train)) + 1

  tf1.disable_v2_behavior()

  ensemble_filenames = []
  for i in range(FLAGS.ensemble_size):
    model = lenet5(x_train.shape[1:], num_classes)

    def negative_log_likelihood(y, rv_y):
      del rv_y  # unused arg
      return -model.output.distribution.log_prob(tf.squeeze(y))  # pylint: disable=cell-var-from-loop

    def accuracy(y_true, y_sample):
      del y_sample  # unused arg
      return tf.equal(
          tf.argmax(input=model.output.distribution.logits, axis=1),  # pylint: disable=cell-var-from-loop
          tf.cast(tf.squeeze(y_true), tf.int64))

    def log_likelihood(y_true, y_sample):
      del y_sample  # unused arg
      return model.output.distribution.log_prob(tf.squeeze(y_true))  # pylint: disable=cell-var-from-loop

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=FLAGS.learning_rate),
        loss=negative_log_likelihood,
        metrics=[log_likelihood, accuracy])
    member_dir = os.path.join(FLAGS.output_dir, 'member_' + str(i))
    tensorboard = tf1.keras.callbacks.TensorBoard(
        log_dir=member_dir,
        update_freq=FLAGS.batch_size * FLAGS.validation_freq)

    if FLAGS.bootstrap:
      inds = np.random.choice(n_train, n_train, replace=True)
      x_sampled = x_train[inds]
      y_sampled = y_train[inds]

    model.fit(
        x=x_train if not FLAGS.bootstrap else x_sampled,
        y=y_train if not FLAGS.bootstrap else y_sampled,
        batch_size=FLAGS.batch_size,
        epochs=(FLAGS.batch_size * FLAGS.training_steps) // n_train,
        validation_data=(x_test, y_test),
        validation_freq=max(
            (FLAGS.validation_freq * FLAGS.batch_size) // n_train, 1),
        verbose=1,
        callbacks=[tensorboard])

    member_filename = os.path.join(member_dir, 'model.weights')
    ensemble_filenames.append(member_filename)
    model.save_weights(member_filename)

  labels = tf.keras.layers.Input(shape=y_train.shape[1:])
  ll = tf.keras.backend.function([model.input, labels], [
      model.output.distribution.log_prob(tf.squeeze(labels)),
      model.output.distribution.logits,
  ])

  ensemble_metrics_vals = {
      'train': ensemble_metrics(
          x_train, y_train, model, ll, weight_files=ensemble_filenames),
      'test': ensemble_metrics(
          x_test, y_test, model, ll, weight_files=ensemble_filenames),
  }

  for split, metrics in ensemble_metrics_vals.items():
    logging.info(split)
    for metric_name in metrics:
      logging.info('%s: %s', metric_name, metrics[metric_name])

if __name__ == '__main__':
  app.run(main)

/usr/local/lib/python3.7/dist-packages/edward2/__init__.py:34: UserWarning: JAX backend for Edward2 is not available.
  warnings.warn("JAX backend for Edward2 is not available.")
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/models.py:39: UserWarning: Skipped due to ImportError: No module named 'official'
  warnings.warn(f'Skipped due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:67: UserWarning: Skipped ViT models due to ImportError: No module named 'flax'
  warnings.warn(f'Skipped ViT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:76: UserWarning: Skipped BERT models due to ImportError: No module named 'official'
  warnings.warn(f'Skipped BERT models due to ImportError: {e}')
/usr/local/lib/python3.7/dist-packages/uncertainty_baselines/models/__init__.py:84: UserWarning: Skipped MIMO models due to ImportError: No module named 'edward2.experim

Instructions for updating:
non-resource variables are not supported in the long term


W0615 09:48:17.935974 140641488009088 deprecation.py:345] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - ETA: 0s - loss: 0.6373 - log_likelihood: -0.6373 - accuracy: 0.7705

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 54s 1ms/sample - loss: 0.6373 - log_likelihood: -0.6373 - accuracy: 0.7705 - val_loss: 0.4337 - val_log_likelihood: -0.4337 - val_accuracy: 0.8454
Epoch 2/25
50000/50000 [==============================] - 54s 1ms/sample - loss: 0.3907 - log_likelihood: -0.3907 - accuracy: 0.8600 - val_loss: 0.3738 - val_log_likelihood: -0.3738 - val_accuracy: 0.8642
Epoch 3/25
50000/50000 [==============================] - 54s 1ms/sample - loss: 0.3340 - log_likelihood: -0.3340 - accuracy: 0.8794 - val_loss: 0.3625 - val_log_likelihood: -0.3625 - val_accuracy: 0.8667
Epoch 4/25
50000/50000 [==============================] - 53s 1ms/sample - loss: 0.3011 - log_likelihood: -0.3011 - accuracy: 0.8904 - val_loss: 0.3226 - val_log_likelihood: -0.3226 - val_accuracy: 0.8793
Epoch 5/25
50000/50000 [==============================] - 55s 1ms/sample - loss: 0.2746 - log_likelihood: -0.2746 - accuracy: 0.8992 - val_loss: 0.2945 - val_log_likelihood: -0.2945 - val_acc

I0615 11:56:50.710220 140641488009088 <ipython-input-9-52e8010f9d94>:172] train
I0615 11:56:50.712752 140641488009088 <ipython-input-9-52e8010f9d94>:174] loss: 0.04672895614873618
I0615 11:56:50.718973 140641488009088 <ipython-input-9-52e8010f9d94>:174] log_likelihood: -0.0467289574444294
I0615 11:56:50.720291 140641488009088 <ipython-input-9-52e8010f9d94>:174] accuracy: 0.983411991596222
I0615 11:56:50.722361 140641488009088 <ipython-input-9-52e8010f9d94>:174] probabilistic_log_likelihood: -0.036324912001165284
I0615 11:56:50.724102 140641488009088 <ipython-input-9-52e8010f9d94>:174] probabilistic_accuracy: 0.99522
I0615 11:56:50.729291 140641488009088 <ipython-input-9-52e8010f9d94>:174] ece: 0.032791690037489246
I0615 11:56:50.732096 140641488009088 <ipython-input-9-52e8010f9d94>:174] mce: 0.23984514954704905
I0615 11:56:50.734789 140641488009088 <ipython-input-9-52e8010f9d94>:174] brier_score: 0.0014048021176784416
I0615 11:56:50.739910 140641488009088 <ipython-input-9-52e8010f9d94>

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
